## Librerias

In [1]:
import pandas as pd
import joblib
import os
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import GridSearchCV, KFold , train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor

os.chdir('../')

/Users/dancastillot/Desktop/DANI/miad_proyecto_despliegue/.conda/lib/python3.9/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/dancastillot/Desktop/DANI/miad_proyecto_despliegue/.conda/lib/python3.9/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


## Datos entrenamiento

In [2]:
df_train = joblib.load("datos/data_train.joblib")


## Encoder y selector de variables

In [3]:
fwiz = joblib.load("modelos/fwiz.joblib")
cat_econder = joblib.load("modelos/cat_econder.joblib")

Imported 0.3.2 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False, skip_xgboost=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



## Aplicar encoder y selector de variables

In [4]:
X_train_t = cat_econder.transform(df_train.drop(columns=['clean_valor_total_avaluo']))

In [5]:
X_train_selected = pd.concat([fwiz.transform(X_train_t), df_train['clean_valor_total_avaluo']], axis=1)

In [6]:

X = X_train_selected.drop('clean_valor_total_avaluo', axis=1)
y = X_train_selected['clean_valor_total_avaluo']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Iterar modelos 

In [7]:
#random forest regressor
# xgboost regressor
#lgbm regressor 
#gradient boosting regressor
#decision tree regressor

In [8]:
# Definir la cuadrícula de parámetros para cada modelo
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
}

param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_lgbm = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_dt = {
    'max_depth': [5, 10, 20],
}

In [9]:
from sklearn.metrics import  make_scorer

# Import the required modules
from sklearn.metrics import make_scorer
import os

# Define the log_model function
def log_model(model, model_name):
    mlflow.sklearn.log_model(model, model_name)

# Define the run_experiment function
def run_experiment(model, param_grid, model_name, run_name):
    with mlflow.start_run(run_name=run_name):
        # Define the k-fold cross-validation
        cv = KFold(n_splits=5, shuffle=True, random_state=42)

        # Use MAPE as the scoring metric for grid search
        scoring = make_scorer(mean_absolute_error, greater_is_better=False)

        # Perform grid search with cross-validation
        grid_search = GridSearchCV(model, param_grid, scoring=scoring, cv=cv)
        grid_search.fit(X, y)
        
        # Get the best parameters and the fitted model
        best_params = grid_search.best_params_
        best_model = grid_search.best_estimator_

        # Calculate the average MAPE using cross-validation
        mape_scores = -grid_search.cv_results_['mean_test_score']
        average_mape = mape_scores.mean()


        # Log the results in MLflow
        mlflow.log_params(best_params)
        mlflow.log_metric('Average_MAPE', average_mape)
        log_model(best_model, model_name)


# Set the custom model path
os.environ["MLFLOW_RESULTS"] = "/Users/dancastillot/Desktop/DANI/miad_proyecto_despliegue"



# Execute experiments for each model
run_experiment(RandomForestRegressor(), param_grid_rf, 'RandomForestRegressor', run_name='experiment_rf')
run_experiment(XGBRegressor(), param_grid_xgb, 'XGBRegressor', run_name='experiment_xgb')
run_experiment(LGBMRegressor(), param_grid_lgbm, 'LGBMRegressor', run_name='experiment_lgbm')
run_experiment(GradientBoostingRegressor(), param_grid_gb, 'GradientBoostingRegressor', run_name='experiment_gb')
run_experiment(DecisionTreeRegressor(), param_grid_dt, 'DecisionTreeRegressor', run_name='experiment_dt')

OSError: [Errno 30] Read-only file system: '/c:'